In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

resnet_model = ResNet50(input_shape=(224,224,3), include_top=False)
resnet_model.trainable = True

model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ ?                           │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [2]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_gen = ImageDataGenerator(rotation_range=20, 
                               width_shift_range=0.2, height_shift_range=0.2,
                               horizontal_flip=True)
train_data = train_gen.flow_from_directory('./glaucoma/train', target_size=(224,224),
                                           batch_size=32, class_mode='sparse')

Found 1394 images belonging to 3 classes.


In [7]:
test_gen = ImageDataGenerator()
test_data = test_gen.flow_from_directory('./glaucoma/test', target_size=(224,224), 
                                         batch_size=32, class_mode='sparse')

Found 150 images belonging to 3 classes.


In [8]:
model.fit(train_data, validation_data=test_data, epochs=20)

Epoch 1/20


C:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


44/44 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.6004 - loss: 15.1824 - val_accuracy: 0.3733 - val_loss: 8896777.0000
Epoch 2/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.6771 - loss: 1.9782 - val_accuracy: 0.5200 - val_loss: 150931.6250
Epoch 3/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.7009 - loss: 1.3882 - val_accuracy: 0.6000 - val_loss: 1.0390
Epoch 4/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.7078 - loss: 0.6743 - val_accuracy: 0.6067 - val_loss: 0.8647
Epoch 5/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.7328 - loss: 0.6397 - val_accuracy: 0.6400 - val_loss: 0.8447
Epoch 6/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.7277 - loss: 0.6037 - val_accuracy: 0.6533 - val_loss: 0.8324
Epoch 7/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.7191 - loss: 0.6152 - val_accuracy: 0.6467 - val_loss: 0.7719
Epoch 8/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.6940 - loss: 0.6271 - val_accuracy: 0.6267 - val_los

In [9]:
model.save('glaucoma_model.keras')

In [10]:
from tensorflow.keras.models import load_model
model2 = load_model('glaucoma_model.keras')
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │     102,761,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           3,075 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 378,950,539 (1.41 GB)

 Trainable params: 126,299,139 (481.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

 Optimizer params: 252,598,280 (963.59 MB)

In [17]:
from tensorflow.keras.preprocessing import image
img = image.load_img('./glaucoma/test/advanced_glaucoma/1.png', target_size=(224,224))
x = image.img_to_array(img).reshape(-1, 224, 224, 3)

In [18]:
pred = model2.predict(x)
print(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[[9.5729983e-01 4.2133801e-02 5.6639727e-04]]


In [19]:
import numpy as np
print(np.argmax(pred, axis=1))

[0]


In [20]:
print(test_data.classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [21]:
y_preds = model2.predict(test_data)
y_pred = np.argmax(y_preds, axis=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 633ms/step


In [22]:
import pandas as pd
pd.crosstab(test_data.classes, y_pred)

col_0,0,1,2
row_0,,,
0,22,4,20
1,12,3,11
2,39,8,31


In [23]:
model2.evaluate(test_data)

5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 305ms/step - accuracy: 0.7697 - loss: 0.6443


[0.6953138113021851, 0.7400000095367432]